In [22]:
import netCDF4
import os
import csv
import datetime as dt
from netCDF4 import num2date
import netCDF4 as nc
from datetime import date
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from pprint import pprint
from matplotlib import pyplot as plt

import praw  # Reddit API
import nltk  # for NLP
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SNTA

pd.set_option('max_columns', 100)
pd.set_option('max_rows', 500)

# Import the companies dataframe

In [9]:
df_companies = pd.read_csv('constituents.csv')
df_companies

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


# Import and organize post and comment DataFrames

In [10]:
path = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\downloaded data\\' 
output_posts = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\downloaded data\summaries\posts\\'

In [11]:
subreddit = 'WallStreetBets'  # subreddit for analysis
start_year = 2022  # set starting year of analysis
end_year = 2022  # set ending year of the analysis (included in the analysis)

files = os.listdir(path+str(end_year)+'\\'+subreddit+'\\'+'posts\\')
files

['WallStreetBets-1-5-2022-submissions.csv',
 'WallStreetBets-10-4-2022-submissions.csv',
 'WallStreetBets-11-4-2022-submissions.csv',
 'WallStreetBets-16-4-2022-submissions.csv',
 'WallStreetBets-17-4-2022-submissions.csv',
 'WallStreetBets-18-4-2022-submissions.csv',
 'WallStreetBets-19-4-2022-submissions.csv',
 'WallStreetBets-2-5-2022-submissions.csv',
 'WallStreetBets-3-5-2022-submissions.csv',
 'WallStreetBets-4-5-2022-submissions.csv',
 'WallStreetBets-5-5-2022-submissions.csv',
 'WallStreetBets-6-5-2022-submissions.csv',
 'WallStreetBets-7-5-2022-submissions.csv',
 'WallStreetBets-8-5-2022-submissions.csv',
 'WallStreetBets-9-5-2022-submissions.csv']

Generate DataFrame with all the submissions

In [12]:
df_submissions = pd.DataFrame()
subreddit = 'WallStreetBets'  # subreddit for analysis
start_year = 2022  # set starting year of analysis
end_year = 2022  # set ending year of the analysis (included in the analysis)

for year in range(start_year, end_year+1):  # 2021 and 2022 for now
    files = os.listdir(path+str(year)+'\\'+subreddit+'\\'+'posts\\')
    for file in files:
        temp_df = pd.read_csv(path+str(year)+'\\'+subreddit+'\\'+'posts\\'+file)
        df_submissions = pd.concat([df_submissions, temp_df], axis=0, ignore_index=True)

    
# unix to datetime
df_submissions['DateTime'] = pd.to_datetime(df_submissions['created_utc'],
                                            unit='s', 
                                            errors='coerce'
                                           )

df_submissions.sort_values(by='DateTime', ascending=True, inplace=True)

# df_submissions.reset_index(inplace=True)

df_submissions.to_csv(output_posts+'submissions.csv', 
                      index=False
                     )
df_submissions

,id,url,title,score,upvote_rate,num_comments,created_utc,selftext,DateTime
727,u04so3,https://i.redd.it/3pcb2n5s5ls81.jpg,"“It’s Chess, not Checkers”",35,0.82,7,1.649546e+09,NaN,2022-04-09 23:14:07
726,u04zf6,https://www.reddit.com/gallery/u04zf6,played Tesla options last week. 5k to 17k,381,0.94,75,1.649547e+09,NaN,2022-04-09 23:24:20
725,u050n9,https://i.redd.it/bs42jwrx7ls81.jpg,Apes all over the shop... You’ll be one of the...,77,0.83,29,1.649547e+09,NaN,2022-04-09 23:26:12
724,u051yw,https://gfycat.com/lividliveboa,Wen Moon?,11,0.88,1,1.649547e+09,NaN,2022-04-09 23:28:22
723,u0540r,https://i.redd.it/8zrcapvu8ls81.jpg,S&P Futures Technical Analysis by Adam Mancini,1,1.00,1,1.649547e+09,[deleted],2022-04-09 23:31:21
...,...,...,...,...,...,...,...,...,...
6695,um1y9b,https://imgur.com/4oCp800.gif,Me after going all in oil stocks in mid 2020 b...,1,1.00,0,1.652132e+09,[deleted],2022-05-09 21:33:30
6694,um1yb1,https://i.redd.it/9hdug9j4riy81.jpg,THE HOUSING MARKET IS GOING TO ZERO LIKE IN 20...,1,1.00,1,1.652132e+09,[deleted],2022-05-09 21:33:33
6693,um1zbn,https://i.imgur.com/tFcgybb.jpeg,My portfolio after today,1,1.00,0,1.652132e+09,NaN,2022-05-09 21:34:50
6692,um201i,https://i.redd.it/616q1v6griy81.gif,"Me, knowing nothing about investing, going all...",2,1.00,2,1.652132e+09,NaN,2022-05-09 21:35:42


Generate DataFrame with all the comments

In [18]:
output_comments = r'C:\Users\Panagiotis.Mavritsak\Desktop\Reddit sentiment analysis\downloaded data\summaries\comments\\'

In [34]:
df_comments = pd.DataFrame()
temp_df = pd.DataFrame()
subreddit = 'WallStreetBets'  # subreddit for analysis
start_year = 2022  # set starting year of analysis
end_year = 2022  # set ending year of the analysis (included in the analysis)

for year in range(start_year, end_year+1):  # 2021 and 2022 for now
#     temp_submissions_df = pd.read_csv(path+str(year)+'\\'+subreddit+'\\'+str(year)+'-WallStreetBets-submissions.csv')
#     for id in temp_submissions_df['id']:
#         temp_df = pd.read_csv(path+str(year)+'\\'+subreddit+'\\'+str(year)+'-WallStreetBets-submission_'+id+'-comments.csv')
#         df_comments = pd.concat([df_comments, temp_df], axis=0, ignore_index=True)


# Alternative way by reading all files there
    files = os.listdir(path+str(year)+'\\'+subreddit+'\\'+'comments\\')
    for file in files:
        temp_df = pd.read_csv(path+str(year)+'\\'+subreddit+'\\'+'comments\\'+file)
        df_comments = pd.concat([df_comments, temp_df], axis=0, ignore_index=True)

for i in range(len(df_comments)):  # keep the pure id of the initial post
    df_comments['comment_parent_id'][i] = df_comments['comment_parent_id'].values[i][3:]

df_comments.to_csv(output_comments+'comments.csv', 
                   index=False
                  )
df_comments

,comment_id,comment_parent_id,comment_body,comment_link_id
0,i43gb7l,u04so3,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,t3_u04so3
1,i43gqan,u04so3,![img](emote|t5_2th52|4271),t3_u04so3
2,i43htzd,u04so3,"It’s Rally’s, not Checkers",t3_u04so3
3,i43h3ho,u04so3,How much did Team Pelosi make on Zoom stock du...,t3_u04so3
4,i43k3y5,i43htzd,Sir this is a Wendy's.,t3_u04so3
...,...,...,...,...
476504,i7yzopa,um1yb1,I am a bot from /r/wallstreetbets. You submitt...,t3_um1yb1
476505,i7yzv6d,um201i,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,t3_um201i
476506,i7yzu0m,um201i,I am a bot from /r/wallstreetbets. You submitt...,t3_um201i
476507,i7yzz1z,um20fo,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,t3_um20fo
